In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm

In [ ]:
# get the metadata
md = pd.read_excel('/home/jvdzwaan/ownCloud/Shared/OCR/Selection_OCR_GT_metadata.xlsx')
md

In [ ]:
def convert_identifier(identifier):
    parts = identifier.split(':')
    return '_'.join([parts[0].upper(), parts[1]])

convert_identifier('ddd:010010081:mpeg21')

In [ ]:
%%time
import os
import glob

in_dir = '/home/jvdzwaan/data/kb-ocr/text_aligned_blocks-match_gs/aligned/'

not_found = 0

for ident in md['Identifier']:
    #print(ident)
    #print(convert_identifier(ident))
    c = convert_identifier(ident)
    g = '{}{}*'.format(in_dir, c)
    files = glob.glob(g)
    if len(files) == 0:
        not_found += 1
    elif len(files) == 1:
        pass
    else:
        print('Trouble with', identifier)

In [ ]:
not_found

In [ ]:
md['file_prefix'] = md.apply(lambda row: convert_identifier(row['Identifier']), axis=1)
md

In [ ]:
def file_available(file_prefix, indir):
    g = '{}{}*'.format(in_dir, file_prefix)
    files = glob.glob(g)
    if len(files) == 0:
        return False
    elif len(files) == 1:
        return True
    else:
        print('Trouble with', file_prefix)
        return 'ERROR'

in_dir = '/home/jvdzwaan/data/kb-ocr/text_aligned_blocks-match_gs/aligned/'
md['file_available'] = md.apply(lambda row: file_available(row['file_prefix'], in_dir), axis=1)
md

In [ ]:
import os

def file_name(file_prefix, indir):
    g = '{}{}*'.format(in_dir, file_prefix)
    files = glob.glob(g)
    if len(files) == 0:
        return 'UNKNOWN'
    elif len(files) == 1:
        return os.path.basename(files[0])
    else:
        print('Trouble with', file_prefix)
        return 'ERROR'

in_dir = '/home/jvdzwaan/data/kb-ocr/text_aligned_blocks-match_gs/aligned/'
md['file_name'] = md.apply(lambda row: file_name(row['file_prefix'], in_dir), axis=1)
md

In [ ]:
import datetime

def get_period(year):
    #print(type(year))
    if isinstance(year, datetime.datetime):
        year = int(year.strftime("%Y"))
    elif isinstance(year, str):
        year = int(year)

    if year < 1883:
        return 1
    elif year >= 1883 and year < 1947:
        return 2
    else:
        return 3

md['period'] = md.apply(lambda row: get_period(row['Date of publication']), axis=1)
md

In [ ]:
def get_year(year):
    #print(type(year))
    if isinstance(year, datetime.datetime):
        year = int(year.strftime("%Y"))
    elif isinstance(year, str):
        year = int(year)
        
    return year

md['year'] = md.apply(lambda row: get_year(row['Date of publication']), axis=1)
md

In [ ]:
md = md.query('file_available == True').copy()
md

In [ ]:
md.columns

In [ ]:
md.to_csv('/home/jvdzwaan/data/kb-ocr/metadata.csv')

In [ ]:
grouped = md[['Date of publication', 'file_prefix', 'file_name', 'ABBYY Version', 'period']].groupby(['ABBYY Version', 'period'])

In [ ]:
grouped.count()

In [ ]:
def create_datadivision(in_files, SEED=4, TEST_PERCENTAGE=10, VAL_PERCENTAGE=10):
    np.random.seed(SEED)
    np.random.shuffle(in_files)

    n_test = int(len(in_files)/100.0 * TEST_PERCENTAGE)
    n_val = int(len(in_files)/100.0 * VAL_PERCENTAGE)

    validation_texts = in_files[0:n_val]
    test_texts = in_files[n_val:n_val+n_test]
    train_texts = in_files[n_val+n_test:]

    division = {
        'train': [os.path.basename(t) for t in train_texts],
        'val': [os.path.basename(t) for t in validation_texts],
        'test': [os.path.basename(t) for t in test_texts]
    }
    
    return division

in_files = list(grouped.get_group((8.1, 1))['file_name'])
print(len(in_files))
division = create_datadivision(in_files)

print('# train', len(division['train']))
print('# val', len(division['val']))
print('# test', len(division['test']))

In [ ]:
# make a datadivision for all periods and Abby 8
import json

for p in (1, 2, 3):
    print('Period', p)
    in_files = list(grouped.get_group((8.1, p))['file_name'])
    print(len(in_files))
    division = create_datadivision(in_files)

    print('# train', len(division['train']))
    print('# val', len(division['val']))
    print('# test', len(division['test']))

    out_file = f'/home/jvdzwaan/data/kb-ocr/text_aligned_blocks-match_gs/datadivision-A8P{p}.json'
    print(out_file)
    with open(out_file, 'w') as f:
        json.dump(division, f, indent=4) 
    print()